In [ ]:
import sys
import json
import nltk
import operator
from collections import Counter

In [ ]:
def read_tweets_from_json(filename):
    tweets=[]
    with open(filename, encoding="utf-8") as file:
        data=json.load(file)
        for tweet in data:
            tweets.append(tweet["text"])
    return tweets

In [ ]:
trump_tweets=read_tweets_from_json("../data/trump_tweets.json")

In [ ]:
aoc_tweets=read_tweets_from_json("../data/aoc_tweets.json")

Explore your assumptions between the words you think will most distinguish the tweets of Donald Trump from those Alexandria Ocasio-Cortez.  Before looking at the data, what words do you think will be comparatively distinct to both?  (If you're not familiar with either, see http://twitter.com/realDonaldTrump and http://twitter.com/AOC).

In [ ]:
def convert_tweets_to_tokens(tweets):
    tokens=[]
    for tweet in tweets:
        tokens.extend(nltk.casual_tokenize(tweet))
    return tokens

In [ ]:
def get_counts(tokens):
    counts=Counter()
    for token in tokens:
        counts[token]+=1
    return counts

The $\chi^2$ test as used in the comparison of different texts is designed to measure how statistically significant the distriubtion of counts in a 2x2 contingency table is.  Use the following function to analyze the difference between these accounts.  How do the most distinct terms comport with your assumptions?

In [ ]:
def chi_square(one_counts, two_counts):

    one_sum=0.
    two_sum=0.
    vocab={}
    for word in one_counts:
        one_sum+=one_counts[word]
        vocab[word]=1
    for word in two_counts:
        vocab[word]=1
        two_sum+=two_counts[word]

    N=one_sum+two_sum
    vals={}
    
    for word in vocab:
        O11=one_counts[word]
        O12=two_counts[word]
        O21=one_sum-one_counts[word]
        O22=two_sum-two_counts[word]
        
        # We'll use the simpler form given in Manning and Schuetze (1999) 
        # for 2x2 contingency tables: 
        # https://nlp.stanford.edu/fsnlp/promo/colloc.pdf, equation 5.7
        
        vals[word]=(N*(O11*O22 - O12*O21)**2)/((O11 + O12)*(O11+O21)*(O12+O22)*(O21+O22))
        
    sorted_chi = sorted(vals.items(), key=operator.itemgetter(1), reverse=True)
    one=[]
    two=[]
    for k,v in sorted_chi:
        if one_counts[k]/one_sum > two_counts[k]/two_sum:
            one.append(k)
        else:
            two.append(k)
    
    print ("@realdonaldtrump:\n")
    for k in one[:20]:
        print("%s\t%s" % (k,vals[k]))

    print ("\n\n@AOC:\n")
    for k in two[:20]:
        print("%s\t%s" % (k,vals[k]))

In [ ]:
trump_tokens=convert_tweets_to_tokens(trump_tweets)
trump_counts=get_counts(trump_tokens)

In [ ]:
aoc_tokens=convert_tweets_to_tokens(aoc_tweets)
aoc_counts=get_counts(aoc_tokens)

In [ ]:
chi_square(trump_counts, aoc_counts)

We saw earlier that $\chi^2$ is not a perfect estimator since it doesn't account for the burstiness of language (the tendency of mentions of the same word to clump together in a text).  Do you expect this to still hold on Twitter?  Why or why not?  How are the differences identified by a $\chi^2$ similar to those by Mann-Whitney?